In [3]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf6248971aa72d85c54387b65a72d6e34b294f')

## Ablauf/ Overview

#### Regen
- Isochronen in 15Minuten-Wege-Abständen, an diesen Punkten (= Kreisen) Wetter abfragen. Pro Isochrone müssen mehrere Punkte ausgewählt werden (Anzahl abhängig vom Radius der jeweiligen Isochrone)
- Erstmal als Vereinfachung: keine tatsächlichen Isochronen verwenden sondern einfach Kreise mit festgelegten Radien (so vermeiden wir eine API-Abfrage). (Aber super fancy).


- Anzahl der Punkte auf die Ränder der jeweiligen Isochrone legen, sodass diese gesamt abgedeckt ist (Umsetzung tbd), dort dann das Wetter abfragen, in abhängigkeit des bereits zurückgelegten Weges

- Ausgabe sind dann x Punkte, mit Regen Ja/Nein, um diese ziehen wir Radius mit festgelegter Größe (vereinfachte Annahme)
#### POI
- diese Punkte + deren zugehöriger Bereich bilden dann die Ausschluss-Areas für die nachfolgende POI-Abfrage.

- Die POI-Abfrage liefert dann nur Orte, wo die Sonne scheint wenn man ankommt.

- User wählt Ziel aus.

#### Routenberechnung
- möglicherweise ist die Route dann länger als vorher angegeben (da Luftlinie für POI verwendet wird). + Dann noch bei Routenwechsel wegen Regen.
- für die Routenberechnung geben wir dann wieder die Regen-Auschluss-Polygone ein.
- fertig.


In [4]:
# user input:

print("How far would you like to travel today? Please input your answer in kilometres.")

distance_input = 0.5 # [km]

How far would you like to travel today? Please input your answer in kilometres.


## Calculate Points of interest within range

In [113]:
homebase = [10.5288243, 52.2733412]

m = folium.Map(location=list(reversed(homebase)), tiles='cartodbpositron', zoom_start=13)

In [105]:
# Coordinates of the university:
geojson = {"type": "point", "coordinates": homebase}


# hier polygone uebergeben, in denen es regnet.

pois = client.places(
    request='pois',
    geojson=geojson,
    buffer= distance_input*1000,  # searches within 500 meters of specified point [m]
    filter_category_ids=[237,240],#[220, 568],  # https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml
    #filter_category_group= ['historic'],
    validate=False,
#     dry_run=True
)

## Output POIs

In [114]:
print("We have found the following destinations:")


for i, poi in enumerate(pois['features']):
    poi_name = poi['properties'].get('osm_tags', {}).get('name', "Kein Name zugeordnet.")
    poi_distance = poi['properties']['distance']
    poi_category_group = poi['properties'].get('category_ids',{}).values().__iter__().__next__()['category_group']

    print(f"{i+1}. : {poi_name}, Distance: {round(poi_distance/1000,2)}km")

    # depending on the category of the POI, change the icon of the marker. examples:
    # list of all icons: https://getbootstrap.com/docs/3.3/components/ (but more with the prefix 'fa' on here: https://fontawesome.com/icons?d=gallery, only works with fa-4!)
    if poi_category_group == 'historic':
        icon = folium.Icon(color="red",icon="bank", prefix='fa')#folium.Icon(prefix='fa', icon='landmark')
    elif poi_category_group == 'natural':
        icon = folium.Icon(color="green",icon="leaf", prefix='fa')#folium.Icon(prefix='fa', icon='landmark')
    else:
        icon = folium.Icon(color='blue')

    folium.Marker(
        location=list(reversed(poi['geometry']['coordinates'])),
        icon = icon,
        popup=folium.Popup(poi_name),
        tooltip=i+1
    ).add_to(m)
m

We have found the following destinations:
1. : Gauß-Denkmal, Distance: 0.45km
2. : Gedenksäule für das Reserve-Infanterie-Regiment 273, Distance: 0.37km
3. : Kein Name zugeordnet., Distance: 0.17km
4. : Kein Name zugeordnet., Distance: 0.37km


In [115]:
print("Where would you like to go? Choose one.")
# user input
chosen_poi_id = 1 # convert that again to zero-based.

chosen_poi = pois['features'][chosen_poi_id - 1]


poi_name = chosen_poi['properties'].get('osm_tags', {}).get('name', "Kein Name zugeordnet.")
poi_distance = chosen_poi['properties']['distance']

poi_coordinates = chosen_poi['geometry']['coordinates']
print(f"You have chosen {poi_name}. Good choice!")

Where would you like to go? Choose one.
You have chosen Gauß-Denkmal. Good choice!


In [116]:
# Now calculate a route to this spot with restrictions on the rainy areas.


In [122]:
# API Call for the route
# TODO: consider rain-polygons

coordinates = [homebase, poi_coordinates]

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)


In [123]:
folium.PolyLine(
        locations=[list(reversed(coord))
                           for coord in
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

# add marker for homebase.
folium.Marker(
        location = list(reversed(homebase)),
        icon = folium.Icon(color="green")
    ).add_to(m)

m